# RSA cryptosystem
Intuition: it is feasible to find integers $d, e, n$ such that
$$ (m^e)^d\equiv m \pmod{n}$$
for integers $m<n$, such that it is infeasible to find $d$ given $e$, $n$, and/or $m$, and that
$$ (m^d)^e\equiv m \pmod{n}.$$


In [12]:
from Crypto.Util.number import getPrime, getRandomInteger, size, inverse
import secrets
import math

In [13]:
def new_key(key_bits:int, e:int = 65537):
    # note: primes are close for Fermat
    p = getPrime(key_bits//2)
    q = getPrime(key_bits//2)

    # public key:
    n = p*q    
    
    # totient function
    phi = (p-1)*(q-1)
    # private key:
    d = inverse(e, phi)

    return(e, n, d)

In [14]:
def encrypt(plaintext:int, exponent:int, public_key:int) -> int:

    ciphertext = pow(plaintext, exponent, public_key)
    
    return ciphertext

In [15]:
def decrypt(ciphertext:int, private_key:int, public_key:int) -> int:

    plaintext = pow(ciphertext, private_key, public_key)
    
    return plaintext

In [16]:
def new_flag(flag_bytes:int):
    with open('/usr/share/dict/words') as w:
		
        numWords = flag_bytes
        words = [word.strip() for word in w]

        flag = ' '.join(secrets.choice(words) for i in range(numWords))
    return flag[0:flag_bytes]

In [17]:
key_bits = 256
(e,n,d) = new_key(key_bits)
flag_bytes = 30
flag = new_flag(flag_bytes)

# encode the string as an integer
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')
ciphertext = encrypt(f, e, n)

p = decrypt(ciphertext, d, n)
# decode the plaintext as string
plaintext = p.to_bytes(key_bits//8, byteorder='big').decode('ascii')
print(f)
print(p)

575768359386974321761174856632276695287233838083748033007141213954404204
575768359386974321761174856632276695287233838083748033007141213954404204


## Special cases

In [20]:
print(encrypt(0,e,n))
print(encrypt(1,e,n))
print(encrypt(n,e,n))
print(encrypt(-1,e,n))
print(encrypt(n-1,e,n))

0
1
0
41205208645688152784868602166305371145094014618651799722270182400743202058076
41205208645688152784868602166305371145094014618651799722270182400743202058076


## Attacks
---
### Small exponent $e$  - plaintext recovery
Suppose $e=3$. For a small plaintext $m$, it is possible to brute-force guess $m$: $$\sqrt[e]{c}$$

In [21]:
import gmpy2
message = 'secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c = pow(m, 3, n)
print(f'modulus: {n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

modulus: 41205208645688152784868602166305371145094014618651799722270182400743202058077
ciphertext: 2042548109113812252163525474272795974844736
cube root(c): (mpz(126879297332596), mpz(0))
guess: 126879297332596
original m: 126879297332596


If $e$ is small but $m^3>n$, it may be possible to recover $m$ from small multiples: $$\sqrt[e]{c+kn}$$

In [25]:
message = 'supr secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c_before_modulo = pow(m,3)
print(f'pow(m,3) before modulo: (unkown without private key) \n{c_before_modulo}')
c = pow(m, 3, n)
print(f'modulus: \n{n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

for k in range(100):
    guess = int(gmpy2.iroot_rem(c+k*n,3)[0])
    confirmed = pow(guess, 3, n) == c
    if confirmed:
        print(f'match found for k={k}')
        break

pow(m,3) before modulo: (unkown without private key) 
2719439991958273208733653161439045609027040839550764719847986258704299828330816
modulus: 
41205208645688152784868602166305371145094014618651799722270182400743202058077
ciphertext: 41101429988543277717194020629196484595929889338397737900424402655991694555811
cube root(c): (mpz(34510584160981754836525344), mpz(3517468179058119397538386791491138412385816586648227))
guess: 34510584160981754836525344
original m: 139581060393214157926851956
match found for k=65


---
## Factoring $n$ - private key recovery
_Factoring attack._

The private key $d$ is constructed from the prime factors of $n$ via the totient function:
$$
\phi(n) = (p-1)\cdot(q-1)\\
d = e^{-1}\pmod{\phi(n)}
$$
Factoring $n$ allows immediate computation of $d$.

Integer factoring options: look-up or computation
- [factordb](http://factordb.com/) or [wolframalpha](https://www.wolframalpha.com/input/?i=factor+)
- [pyecm](https://github.com/martingkelly/pyecm) using [gmpy2](https://github.com/aleaxit/gmpy)

In [26]:
import pyecm

P = getPrime(32)
Q = getPrime(32)
N = P*Q
verbose = False
random_sigma = True
asymptotic_speed = 7
processing_power = 1.0
lf = list(pyecm.factors(N, verbose, random_sigma, asymptotic_speed, processing_power))
print(f'factors of {N}: prod {lf} = {math.prod(lf)}')

factors of 10058500237572902651: prod [mpz(2682566749), mpz(3749580599)] = 10058500237572902651


Factorization difficulty depends on the best known algorithm. For recommendations, see [SP 800-56b](https://csrc.nist.gov/publications/detail/sp/800-56b/rev-2/final).

---
## Common factor
_Factoring attack._

Suppose moduli $n_1,n_2$ were generated with one prime factor $p$ in common; then $p=\gcd{n_1,n_2}$.

$\gcd$ is an extremely quick computation; large keys offer no protection.

See [Lenstra et al](https://eprint.iacr.org/2012/064) _"Ron was wrong, Whit is right"_ and [Heninger et al.](https://www.usenix.org/conference/usenixsecurity12/technical-sessions/presentation/heninger) _"Mining Your Ps and Qs: Detection of Widespread Weak Keys in Network Devices"_.

In [27]:
p = getPrime(2048)
q = getPrime(2048)
r = getPrime(2048)
n1 = p*q
n2 = p*r
print(f'gcd(m,n) = \n{math.gcd(n1,n2)}')
print(f'check: p = \n{p}')

gcd(m,n) = 
25743075022381426310546765828422840203465425132457583337964802151652329705521551864298594809884785410914128599886058194732971314184677536248772953349949418255240569855385136973659103091660457207173591790856090624914505222792747013180937453294830482972538047014139910620790345986587351913718164700972859642919623380987338263971027963668451764075395124581987678259424783993300807321150636641791572942224478136130329572672934124000008194533762003192329192034321863387542308915436850598446811817608983894908991408596545840855833131625856772039717675632536728283205788224643557534059205790537255306281583689132855681604319
check: p = 
2574307502238142631054676582842284020346542513245758333796480215165232970552155186429859480988478541091412859988605819473297131418467753624877295334994941825524056985538513697365910309166045720717359179085609062491450522279274701318093745329483048297253804701413991062079034598658735191371816470097285964291962338098733826397102796366845176407539512458198767

---
## Håstad broadcast
_Small $e$ attack._




In [29]:
# set-up: operations performed by the sender

flag_bytes = 32
flag = new_flag(flag_bytes)
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')

e=3
recipients = 3
key_bits = 1024

n = []
c = []

for i in range(recipients):
    (_,modulus,_) = new_key(key_bits, e=e)
    n.append(modulus)
    ciphertext = encrypt(f, e, modulus)
    c.append(ciphertext)

In [30]:
# attack: operations performed after intercepting ciphertexts c_i, knowing public keys n_i
# apply CRT, take e-th root
from gmpy2 import iroot_rem
N = math.prod(n)
x = 0

for i in range(recipients):
    Ni = N//n[i]
    Mi = inverse(Ni, n[i])
    x += c[i]*Ni*Mi
x %= N
r = iroot_rem(x,e)

In [87]:
# check:
print(f'computed plaintext: {r[0]}')
print(f'original plaintext: {f}')

computed plaintext: 44951042262399525090548103550261314482125373662639964963746244977282326538099
original plaintext: 44951042262399525090548103550261314482125373662639964963746244977282326538099


---
## malleability
_Oracle attack_

In [31]:
# set-up
key_bits = 256
(e,n,d) = new_key(key_bits)
flag_bytes = 30
flag = new_flag(flag_bytes)
f = int.from_bytes(bytes(flag, 'utf-8'), byteorder='big')
ciphertext = encrypt(f, e, n)

def oracle_decrypt(ciphertext):
    plaintext = decrypt(ciphertext, d, n)
    if plaintext != f:
        return plaintext
    else:
        return None

In [33]:
# attack
blind = getRandomInteger(20)
blind_ciphertext = encrypt(blind, e, n)*ciphertext
prophecy = oracle_decrypt(blind_ciphertext)

if prophecy is not None:
    recovered_plaintext = inverse(blind,n)*prophecy%n
    print(f'recovered plaintext: {recovered_plaintext}')
    print(f'original plaintext: {f}')
    # decode the plaintext as string
    p = recovered_plaintext.to_bytes(key_bits//8, byteorder='big').decode('utf-8')
    print(flag)
    print(p)

recovered plaintext: 693017580496240904647204858270188771585302328050680973298406225795642981
original plaintext: 693017580496240904647204858270188771585302328050680973298406225795642981
disbursed Vicky slavishly adve
  disbursed Vicky slavishly adve


---
## LSB oracle
_Oracle attack_

In [34]:
# set-up

def oracle_lsb(c):
    m = decrypt(c, d, n)
    lsb = m%2
    return lsb

key_bits = 256
(e,n,d) = new_key(key_bits)
flag_bytes = 30
flag = new_flag(flag_bytes)
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')
c = encrypt(f, e, n)

In [36]:
# attack

gmpy2.get_context().precision=key_bits+1
U = gmpy2.mpfr(n)
L = 0

for i in range(1, key_bits+1):
    g = encrypt(2**i, e, n)
    o = oracle_lsb(g*c%n)

    if o == 0:
        U = (U+L) / 2
    else:
        L = (U+L) / 2

print(f'Final upper bound: \n{U}')
print(f'Final lower bound: \n{L}')
print(f'original flag: {flag}, int: \n{f}')

Final upper bound: 
796840546118453534503139194091874905816964703097372883510291491554816800.067719
Final lower bound: 
796840546118453534503139194091874905816964703097372883510291491554816799.533619
original flag: stuccoing homespun shrillness , int: 
796840546118453534503139194091874905816964703097372883510291491554816800


---
## Bleichenbacher oracle
Detailed walkthrough on [secgroup.dais.unive.it](http://secgroup.dais.unive.it/wp-content/uploads/2012/11/Practical-Padding-Oracle-Attacks-on-RSA.html); useful source on [github.com/duesee/bleichenbacher](https://github.com/duesee/bleichenbacher).

See also [RFC 2313](https://tools.ietf.org/html/rfc2313) PKCS \#1: RSA Encryption Version 1.5, section 8.1.